<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/001_EDA_(Dados_do_Portal_da_cidad%C3%A2nia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Em construção - Notebook ainda não concluído



In [6]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1 - Carrega o dataset do link externo e copia o arquivo localmente

In [7]:
#Copia os dados das NFEs do portal da cidadância pro drive virtual.
import requests  
file_url = "https://raw.githubusercontent.com//mfilipak/AFRAC_IA/main/DATASET/202201_NFe_NotaFiscalItem.zip"
r = requests.get(file_url, stream = True) 

with open("portal.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)

DATA_FILE = "portal.zip"

#2 - Carrega o dataframe completo em df

In [14]:
df = pd.read_csv(DATA_FILE, encoding="CP1252",sep=";")
print("O dataframe completo contém:",len(df),"linhas")

O dataframe completo contém: 324056 linhas


In [15]:
df[:1]

,CHAVE DE ACESSO,MODELO,SÉRIE,NÚMERO,NATUREZA DA OPERAÇÃO,DATA EMISSÃO,CPF/CNPJ Emitente,RAZÃO SOCIAL EMITENTE,INSCRIÇÃO ESTADUAL EMITENTE,UF EMITENTE,...,PRESENÇA DO COMPRADOR,NÚMERO PRODUTO,DESCRIÇÃO DO PRODUTO/SERVIÇO,CÓDIGO NCM/SH,NCM/SH (TIPO DE PRODUTO),CFOP,QUANTIDADE,UNIDADE,VALOR UNITÁRIO,VALOR TOTAL
0,42220107432517001847550030005499571000224173,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549957,Outra saida merc./prest.serv. nao especif.,01/01/2022 00:02:22,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,257477446,SC,...,0 - NÃO SE APLICA,1,HP RESERVATORIO DE RESIDUO DE TONER,84439939,Outras partes e acessórios para aparelhos de f...,6949,"1,00",UNIDAD,"56,49","56,49"


In [11]:
df.keys()

Index(['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO',
       'DATA EMISSÃO', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE',
       'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE',
       'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO',
       'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL',
       'PRESENÇA DO COMPRADOR', 'NÚMERO PRODUTO',
       'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH',
       'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QUANTIDADE', 'UNIDADE',
       'VALOR UNITÁRIO', 'VALOR TOTAL'],
      dtype='object')

# 3 - Limpeza inicial do dataframe.<br>
df2 conterá somente 4 campos (Data, descr, ncm e cfop)
Eliminados itens duplicados (mesma descr, ncm e cfop)

In [13]:
df2 = df[['DATA EMISSÃO','DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'CFOP']]
df2.columns = ["DATA", "DESCR", "NCM", "CFOP"]
df2 = df2.drop_duplicates(subset=["DESCR", "NCM", "CFOP"])
df2

,DATA,DESCR,NCM,CFOP
0,01/01/2022 00:02:22,HP RESERVATORIO DE RESIDUO DE TONER,84439939,6949
1,01/01/2022 00:03:59,MLTD201LXAZ CARTUCHO DE TONER PRETO 20K PAGINAS,84439933,6949
3,01/01/2022 00:12:39,CLTC609SXAZ CARTUCHO DE TONER CIANO 7K PAGINAS,84439933,6949
7,01/01/2022 00:46:32,MLTD203UXAZ CARTUCHO DE TONER PRETO 15K PAGINAS,84439933,6949
8,01/01/2022 00:55:40,CLTC506LXAZ CARTUCHO DE TONER CIANO 3.5K PAGINAS,84439933,6949
...,...,...,...,...
324044,31/01/2022 23:54:45,FAROL DE MILHA ESQUERDO / DIREITO,85149000,5405
324045,31/01/2022 23:54:45,FAROL LADO ESQUERDO/ DIREITO,85149000,5405
324046,31/01/2022 23:54:45,CUBO DE RODA DIANT DIREITO / ESQUERDO,87089529,5405
324048,31/01/2022 23:55:00,POLTRONA PRESIDENTE GIRATORIA,94021000,6102


#4 - Vamos dar uma olhada nas informações de cada coluna

In [17]:
len(df2["CFOP"].unique()),len(df2['NCM'].unique())

(211, 4672)